Librerie varie da installare

In [45]:
#!pip install pandas
#!pip install matplotlib

Inclusione delle librerie utilizzate

In [46]:
import pandas as pd
import os
import glob
import datetime
import numpy as np

Leggo, per esempio, i file del 2015 delle aziende che non sono ancora in bancarotta

In [47]:
# Path of the file, can be changed
PATH = r"C:\Users\Andre\OneDrive - Università degli Studi di Parma\Tirocinio\Dataset\Attive_rielaborate_G\2015"

# Dataset with all the csv files
input_dataset = pd.DataFrame()

# Path of each csv file
csv_files = glob.glob(os.path.join(PATH, "*.csv"))

# Loop over the list of csv files
for f in csv_files:

    # Read each csv file
    partial_dataset = pd.read_csv(f, sep=";", dtype=object, thousands=",", decimal=".")

    # Print the location and filename
    print('Location:', f)
    print('File Name:', f.split("\\")[-1])

    # Append the partial dataset
    input_dataset = pd.concat([input_dataset, partial_dataset])




print("Dimensione dataset: ", input_dataset.shape)

Location: C:\Users\Andre\OneDrive - Università degli Studi di Parma\Tirocinio\Dataset\Attive_rielaborate_G\2015\Copia di Aida_Export_100001_150000_GG.csv
File Name: Copia di Aida_Export_100001_150000_GG.csv
Location: C:\Users\Andre\OneDrive - Università degli Studi di Parma\Tirocinio\Dataset\Attive_rielaborate_G\2015\Copia di Aida_Export_150001_200000_GG.csv
File Name: Copia di Aida_Export_150001_200000_GG.csv
Location: C:\Users\Andre\OneDrive - Università degli Studi di Parma\Tirocinio\Dataset\Attive_rielaborate_G\2015\Copia di Aida_Export_200001_250000_GG.csv
File Name: Copia di Aida_Export_200001_250000_GG.csv
Location: C:\Users\Andre\OneDrive - Università degli Studi di Parma\Tirocinio\Dataset\Attive_rielaborate_G\2015\Copia di Aida_Export_250001_300000_GG.csv
File Name: Copia di Aida_Export_250001_300000_GG.csv
Location: C:\Users\Andre\OneDrive - Università degli Studi di Parma\Tirocinio\Dataset\Attive_rielaborate_G\2015\Copia di Aida_Export_300001_350000_GG.csv
File Name: Copia d

Rimuovo le colonne inutili allo studio

In [48]:
to_keep_columns = ['Ragione sociale', 'Province', 'Number of employees',
                   'Legal Form', 'Legal Status ', 'TOTAL receivables due from shareholders',
                   'Called share capital', 'TOTAL FIXED ASSETS\n', 'TOTAL INTANGIBLE FIXED ASSETS',
                   'TOTAL TANGIBLE FIXED ASSETS', 'TOTAL FINANCIAL FIXED ASSETS', 'TOTAL CURRENT ASSETS',
                   'TOTAL FINANCIAL ASSETS', 'TOTAL ACCRUED INCOME AND PREPAID EXPENSES', 'Total asset',
                   "TOTAL SHAREHOLDERS' FUNDS", 'TOTAL PROVISIONS FOR RISKS AND CHARGES', 'SEVERANCE INDEMNITY RESERVE',
                   'TOTAL PAYABLES', 'TOTAL VALUE OF PRODUCTION', 'TOTAL PRODUCTION COSTS' ,'OPERATING MARGIN',
                   'Added Value', 'TOTAL FINANCIAL INCOME AND CHARGE', 'TOTAL FINANCIAL ASSETS ADJUSTMENTS',
                   'PROFIT/LOSS BEFORE TAXATION', 'Employees']
filtered_dataset = input_dataset[to_keep_columns]

In [49]:
del input_dataset

Cerco valori mancanti

In [50]:
filtered_dataset.replace(to_replace="n.a.", value=np.NaN, inplace=True)
filtered_dataset.replace(to_replace="n.d.", value=np.NaN, inplace=True)
filtered_dataset.replace(to_replace="#VALUE!", value=np.NaN, inplace=True)
filtered_dataset.isna().sum()

Ragione sociale                              53637
Province                                     53690
Number of employees                          69910
Legal Form                                   53756
Legal Status                                 53637
TOTAL receivables due from shareholders      54494
Called share capital                         57613
TOTAL FIXED ASSETS\n                         54494
TOTAL INTANGIBLE FIXED ASSETS                54494
TOTAL TANGIBLE FIXED ASSETS                  54494
TOTAL FINANCIAL FIXED ASSETS                 54494
TOTAL CURRENT ASSETS                         54494
TOTAL FINANCIAL ASSETS                       54494
TOTAL ACCRUED INCOME AND PREPAID EXPENSES    54494
Total asset                                  53637
TOTAL SHAREHOLDERS' FUNDS                    54459
TOTAL PROVISIONS FOR RISKS AND CHARGES       54494
SEVERANCE INDEMNITY RESERVE                  54494
TOTAL PAYABLES                               54494
TOTAL VALUE OF PRODUCTION      

Filtro i valori non disponibili

In [51]:
filtered_dataset = filtered_dataset.dropna()
filtered_dataset.isna().sum()

Ragione sociale                              0
Province                                     0
Number of employees                          0
Legal Form                                   0
Legal Status                                 0
TOTAL receivables due from shareholders      0
Called share capital                         0
TOTAL FIXED ASSETS\n                         0
TOTAL INTANGIBLE FIXED ASSETS                0
TOTAL TANGIBLE FIXED ASSETS                  0
TOTAL FINANCIAL FIXED ASSETS                 0
TOTAL CURRENT ASSETS                         0
TOTAL FINANCIAL ASSETS                       0
TOTAL ACCRUED INCOME AND PREPAID EXPENSES    0
Total asset                                  0
TOTAL SHAREHOLDERS' FUNDS                    0
TOTAL PROVISIONS FOR RISKS AND CHARGES       0
SEVERANCE INDEMNITY RESERVE                  0
TOTAL PAYABLES                               0
TOTAL VALUE OF PRODUCTION                    0
TOTAL PRODUCTION COSTS                       0
OPERATING MAR

In [52]:
print("Dimensione dataset filtrato dai valori mancanti e con numer odi colonne ridotte: ", filtered_dataset.shape)
filtered_dataset.head()

Dimensione dataset filtrato dai valori mancanti e con numer odi colonne ridotte:  (720129, 27)


,Ragione sociale,Province,Number of employees,Legal Form,Legal Status,TOTAL receivables due from shareholders,Called share capital,TOTAL FIXED ASSETS\n,TOTAL INTANGIBLE FIXED ASSETS,TOTAL TANGIBLE FIXED ASSETS,...,SEVERANCE INDEMNITY RESERVE,TOTAL PAYABLES,TOTAL VALUE OF PRODUCTION,TOTAL PRODUCTION COSTS,OPERATING MARGIN,Added Value,TOTAL FINANCIAL INCOME AND CHARGE,TOTAL FINANCIAL ASSETS ADJUSTMENTS,PROFIT/LOSS BEFORE TAXATION,Employees
0,BOWLINGMANIA S.R.L.,Pesaro Urbino,13,One-person company with limited liability - SRL,Active,0,0,"151,396","3,079","148,317",...,"56,895","262,974","888,864","839,723","49,676","256,301",-632,0,"49,044",1
1,BOX & CO. S.R.L.,Siena,3,Limited liability company - SRL,Active,0,0,"53,227","1,280","39,947",...,"14,562","570,611","701,211","641,240","59,971","168,946","-16,128",0,"43,843",4
2,BOX 2000 - S.R.L.,Bologna,0,Limited liability company - SRL,Active,0,0,"2,726",0,"2,726",...,0,"1,065,464","29,881","34,934","-5,053","-5,053",-1,0,"-5,054",0
3,BOX 4.IT S.R.L.,Reggio nell'Emilia,2,Limited liability company - SRL,Active,0,0,"346,294","1,866","344,428",...,"3,099","780,142","774,532","675,414","99,118","228,353","-2,183",0,"96,935",2
4,BOX 48 S.R.L.,Macerata,0,One-person company with limited liability - SRL,Active,0,0,0,0,0,...,0,"2,800","114,903","79,454","35,449",666,-379,0,"35,070",0


Casto le colonne a tipi di dato sensati

In [53]:
str_cols = ['Ragione sociale', 'Province',
            'Legal Form', 'Legal Status ']


int_cols = ['Number of employees', 'Employees']


for col in filtered_dataset.columns:
    if col in str_cols:
        filtered_dataset[col] = filtered_dataset[col].astype(str)
    else:
        filtered_dataset[col] = filtered_dataset[col].str.replace(',','')
        filtered_dataset[col] = pd.to_numeric(filtered_dataset[col])

Sposto le colonne non numeriche

In [54]:
complete_dataset = filtered_dataset.copy()
descriptive_dataset = filtered_dataset[str_cols]
filtered_dataset.drop(str_cols, axis=1, inplace=True)

Guardo le statistiche delle restanti colonne

In [55]:
filtered_dataset.describe(include="all").T

,count,mean,std,min,25%,50%,75%,max
Number of employees,720129.0,4.649777e+00,5.228630e+01,0.000000e+00,0.0,1.0,4.0,1.525400e+04
TOTAL receivables due from shareholders,720129.0,2.730275e+03,1.826454e+05,0.000000e+00,0.0,0.0,0.0,9.238818e+07
Called share capital,720129.0,3.557514e+02,2.218633e+04,0.000000e+00,0.0,0.0,0.0,7.000000e+06
TOTAL FIXED ASSETS\n,720129.0,9.258115e+05,1.907489e+07,0.000000e+00,3380.0,30416.0,237890.0,7.281671e+09
TOTAL INTANGIBLE FIXED ASSETS,720129.0,4.518071e+04,2.001370e+06,0.000000e+00,0.0,659.0,5882.0,8.887432e+08
TOTAL TANGIBLE FIXED ASSETS,720129.0,4.433264e+05,9.469632e+06,0.000000e+00,540.0,12678.0,133177.0,7.091379e+09
TOTAL FINANCIAL FIXED ASSETS,720129.0,4.373044e+05,1.583168e+07,0.000000e+00,0.0,0.0,995.0,5.545405e+09
TOTAL CURRENT ASSETS,720129.0,8.443719e+05,1.472855e+07,0.000000e+00,41673.0,132333.0,382644.0,6.926224e+09
TOTAL FINANCIAL ASSETS,720129.0,3.764562e+04,2.299952e+06,0.000000e+00,0.0,0.0,0.0,8.823475e+08
TOTAL ACCRUED INCOME AND PREPAID EXPENSES,720129.0,1.346811e+04,2.306627e+05,0.000000e+00,0.0,157.0,2133.0,1.071746e+08


In [56]:
descriptive_dataset.describe().T

,count,unique,top,freq
Ragione sociale,720129,520756,AURORA S.R.L.,254
Province,720129,108,Roma,86780
Legal Form,720129,73,S.R.L.,381054
Legal Status,720129,3,Attiva,525712


In [57]:
complete_dataset

,Ragione sociale,Province,Number of employees,Legal Form,Legal Status,TOTAL receivables due from shareholders,Called share capital,TOTAL FIXED ASSETS\n,TOTAL INTANGIBLE FIXED ASSETS,TOTAL TANGIBLE FIXED ASSETS,...,SEVERANCE INDEMNITY RESERVE,TOTAL PAYABLES,TOTAL VALUE OF PRODUCTION,TOTAL PRODUCTION COSTS,OPERATING MARGIN,Added Value,TOTAL FINANCIAL INCOME AND CHARGE,TOTAL FINANCIAL ASSETS ADJUSTMENTS,PROFIT/LOSS BEFORE TAXATION,Employees
0,BOWLINGMANIA S.R.L.,Pesaro Urbino,13,One-person company with limited liability - SRL,Active,0,0,151396,3079,148317,...,56895,262974,888864,839723,49676,256301,-632,0,49044,1
1,BOX & CO. S.R.L.,Siena,3,Limited liability company - SRL,Active,0,0,53227,1280,39947,...,14562,570611,701211,641240,59971,168946,-16128,0,43843,4
2,BOX 2000 - S.R.L.,Bologna,0,Limited liability company - SRL,Active,0,0,2726,0,2726,...,0,1065464,29881,34934,-5053,-5053,-1,0,-5054,0
3,BOX 4.IT S.R.L.,Reggio nell'Emilia,2,Limited liability company - SRL,Active,0,0,346294,1866,344428,...,3099,780142,774532,675414,99118,228353,-2183,0,96935,2
4,BOX 48 S.R.L.,Macerata,0,One-person company with limited liability - SRL,Active,0,0,0,0,0,...,0,2800,114903,79454,35449,666,-379,0,35070,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52049,SANTA RITA SOCIETA' COOPERATIVA,Agrigento,0,S.C.A.R.L.P.A.,Attiva,0,0,104573,0,104573,...,0,106427,0,2111,-2111,-2111,0,0,-2111,0
52050,SUPERCINEMA S.R.L.,Agrigento,0,S.R.L.,Attiva,0,0,0,0,0,...,0,8656,6276,2838,3438,3438,0,0,3438,0
52051,MADONNA DELL'ITRIA SOCIETA' COOPERATIVA AGRICOLA,Agrigento,0,S.C.A.R.L.P.A.,Attiva,0,0,11765,0,11265,...,0,13494,1235,875,360,-875,0,0,360,0
52052,COOPERATIVA UNIONE DI CAMASTRA E NARO - SOCIET...,Agrigento,1,S.C.A.R.L.P.A.,Attiva,605,605,1099848,0,296895,...,13016,116933,-5426,83431,-88857,-42600,66520,-114495,-136832,1
